# Function documentation

In [8]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.12.1-DEV ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


### File: `fileHandling.jl`

**Function:** `string2Int64Vector(s::String)`

Given a string of space-separated integers, returns a vector of those integers. 

*Example:* `string2Int64Vector("1 2 3") == [1,2,3]`

---

**Function:** `vec2String(s::String)`

Given a vector of integers, returns a space-separated string of these integers. 

*Example:* `vec2String([1,2,3]) == "1 2 3" [1,2,3]`

---

**Function:** `file2SetVectors(s::String)`

Given a file where each line is a space-separated list of integers, this returns a `Vector{Vector}{Int64}` where each entry is a 

*Example:* If the file `io.dat` consists of

```
1 2 3
4 5 6
```
then `file2SetVectors(io.dat) == [[1,2,3],[4,5,6]]`. 

---

**Function:** `to_star0(S::Vector, n::Int64)`

given a subset `S` of a set of size  `n`, returns a `*/0` vector where `*` means the element lies in `S`, and `0` otherwise. 
*Example:* 
```
X = collect(1:12)
S = [1,4,6,10]
to_star0(S, 12) == "*00*0*000*00"
```


In [158]:
function string2Int64Vector(s::String)
    return map(i -> parse(Int64, i), split(s))
end

function vec2String(v::Vector{Vector{Int64}})
    vecString = map(i -> string(i), v)
    return join(vecString, " ")
end

function file2SetVectors(fileName::String)
    return map(s -> string2Int64Vector(s), readlines(fileName))
end

function to_star0(S::Vector, n::Int64)
    return join(map(x -> x in S ? "*" : "0", 1:n ))
end

to_star0 (generic function with 2 methods)

### File: `isolate3Lines.jl`

**Function:** `lines(Q::Matroid)`

Given a simple rank 3 matroid `Q`, returns the set of lines $\mathcal{L}(\mathsf{Q})$ (in the simple matroid case, this is the same as the set of rank-3 cyclic flats). 

*Example:*
```
Q = non_fano_matroid()
lines(Q) == [[3,4,7], [2,5,7], [2,4,6], [1,6,7], [1,4,5], [1,2,3]
```
---

**Function:** `count_lines_thru_one_point(Ls::Vector{Int64}, i::Int64)`

Given the set of lines `Ls` and an element of the ground set `i`, returns the number of lines through `i`. 

*Example:*
```
Ls = lines(non_fano_matroid());
count_lines_thru_one_point(Ls, 7) == 3. 
```

---

**Function:** `count_3_lines_thru_all_points(Q::Matroid)`

Given a simple rank-3 matroid `Q`, applies `count_lines_thru_one_point` to all elements of the ground set of `Q`.

*Example:*
```
Q = non_fano_matroid()
count_3_lines_thru_all_points(Q) == [3,3,2,3,2,2,3]
```

---

**Function:** `to_revlex(Q::Matroid, nCd::Vector{Vector{Int64}})`

Given a matroid `Q` and `nCd` the list of `d` element subsets of `1:n` in revlex, returns revlex basis encoding of `Q`. 

*Example:* 
```
Q = non_fano_matroid()
nCd = sort!(subsets(Vector(1:7), 3), by = x->reverse(x))
to_revlex(Q,nCd) == "0******0******0**********0*0**0****"
```

In [14]:
function lines(Q::Matroid)
    return [l for l in hyperplanes(Q) if length(l) > rank(Q)-1 ]
end

function count_lines_thru_one_point(Ls::Vector{Vector{Int64}}, i::Int64)
    return length([l for l in Ls if i in l])
end

function count_3_lines_thru_all_points(Q::Matroid)
    Ls = lines(Q)    
    return [count_lines_thru_one_point(Ls,i) for i in matroid_groundset(Q)]
end

function to_revlex(Q::Matroid, nCd::Vector{Vector{Int64}})
    B = bases(Q)
    l = []
    for b in nCd
        if b in B
            push!(l,"*")
        else
            push!(l,"0")
        end
    end
    return join(l)
end

to_revlex (generic function with 1 method)

### File: `matroid_realization.final.jl`

**Note**: Much of this will be incorporated in a future version of the matroids package in Oscar, where other contributors include Benjamin Schr&ouml;ter and Lukas K&uuml;hne. 

**Structure:** `MatroidRealizationSpace`.

This is a structure that contains the data of a matroid realization space. If the realization space is represented as the ring $S^{-1}B/I$, then

- `defining_ideal` is the ideal $I$.

- `inequations` is a list of the generators of the semigroup $S$.

- `ambient_ring` is the polynomial ring $B$.

- `representation_matrix` is the matrix where the entries are polynomials in $B$ so when you plug in values that satisfy the equations in $I$ and inequations from $S$, the resulting matrix is a realization of the matroid. 

- `representable` returns `true` if the matroid is representable, and `false` if it is not. 

---

**Function:** `poly_2_factors(f::RingElem)`

Given a polynomial, returns the unique non-unit irreducible factors (but not the exponents).

*Example:*
```
R, (x,y,z) = QQ["x", "y", "z"]
f = 2*(x^2+y)^2*(x+y*z)^4
poly_2_factors(f) == [x^2+y, x+y*z]
```
---

**Function:** `gens_2_factors(Sgens::Vector{<:RingElem})`

Given a list of polynomials, returns a list of the unique non-unit irreducible factors of these polynomials (without the exponents). 

*Example:*
```
R, (x,y,z) = QQ["x", "y", "z"]
Sgens = [2*(x^2+y)^2*(x+y*z)^4, 3*x^2*(x+y*z)^5] 
gens_2_factors(Sgens) == [x^2+y, x+y*z, x]

```
---

**Function:** `stepwise_saturation(I::Ideal, Sgens::Vector{<:RingElem})`

Returns the saturation of the ideal `I` by the multiplicative semigroup generated by the `Sgens`. 

*Example:* 
```
R, (x,y,z) = QQ["x", "y", "z"]
I = ideal(R, [x^2*(y+z), y^3*(y+z), z*(y+z)])
Sgens = [x,y,z]
stepwise_saturation(I,Sgens) == ideal(R, [y+z]) 
```
---


**Function:** `projective_identity(d::Int)`

For $d\geq 2$, this returns the $d\times (d+1)$ matrix whose first $d$ columns form the identity matrix, and the last column is the all 1's vector. 

*Example*:
```
projective_identity(3) == [1 0 0 1; 0 1 0 1; 0 0 1 1]
```

---

**Function:** `interlace_columns(M::MatrixElem{<:RingElem}, N::MatrixElem{<:RingElem}, A::Vector{Int}, R::AbstractAlgebra.Ring)   `

Given 2 `R`-valued matrices `M` and `N` with dimensions $m_1 \times n$ and $m_2 \times n$, this returns the $(m_1+m_2)\times n$ matrix where the columns of `M` are in the positions dictated by `A`. 

*Example:*
```
R, (x,y,z) = QQ["x", "y", "z"]
M = matrix(R, [1 0; 0 1])
N = matrix(R, [x y z; x y z])
A = [2, 4]

interlace_columns(M,N,A,R)  == matrix(R, [x 1 y 0 z; x 0 y 1 z])
```
---

**Function:** `realization_bases_coordinates(Bs::Vector{Vector{Int}}, A::Vector{Int})`

Given the bases `Bs` of a matroid $\mathsf{Q}$ and a $\mathsf{rank}(\mathsf{Q})+1$ circuit `A`, returns entries of the matrix whose entries are nonzero. 

*Example:* 
```
Bs = bases(non_fano_matroid())
realization_bases_coordinates(Bs, [1,2,4,7]) == [[1,1],[2,1],[1,2],[3,2],[3,3]]
```
---

**Function:** `partial_matrix_max_rows(Vs::Vector{Vector{Int}})`

`Vs` is a vector of pairs of integers, returns a dictionary `D` where the keys are the values in the 2nd entries, `D[c]` returns the largest `i` such that `[i,c]` is in `Vs`.  

*Example:*
```
Vs = [[1,1],[2,1],[1,2],[3,2],[3,3]]
D = partial_matrix_max_rows(Vs)
D[1] == 2
D[2] == 3
D[3] == 3
```
---

**Function:** ```realization_matrix(Q::Matroid, A::Vector{Int}, F::AbstractAlgebra.Ring)```

Given a matroid `Q`, creates the reference matrix with the projective identity in the columns indexed by `A` and the polynomial ring . 

*Example:* 
```
Q = non_fano_matroid()
R, (x1,x2,x3), M = realization_matrix(Q, [1,2,4,7], QQ)
M == matrix(R, [1 0 x1 0 x2 0 1; 0 1 1 0 0 x3 1; 0 0 0 1 1 1 1])

```
---

**Function:** `new_matroid_realization_space(Q::Matroid, A::Vector{Int}; F::AbstractAlgebra.Ring = ZZ, saturate::Bool=false)`

Creates the realization space of `Q` as a `MatroidRealizationSpace`, where the projective identity is in columns indexed by `A`.

*Example:* 
```
Q = non_fano_matroid()
A = [1, 2, 4, 7]
MR = new_matroid_realization_space(Q, A; F=QQ);
R = MR.ambient_ring; (x1,x2,x3) = gens(R);
I = ideal(R, [x3-1,x2-1,x1-1])
S = [x1*x3 - x1 + 1, x3, x2*x3 - x2 - x3, x1 + x2 - 1, x2, x1, x1*x3 + x2]
(MR.defining_ideal, MR.inequations) == (I, S) 

```

In [70]:
struct MatroidRealizationSpace
    defining_ideal::Union{Ideal, NumFieldOrdIdl, Nothing}
    inequations::Union{Vector{Oscar.RingElem},Nothing}
    ambient_ring::Union{Oscar.MPolyRing, Oscar.Ring, Nothing}
    representation_matrix::Union{Oscar.MatElem,Nothing}
    representable::Bool
end

function Base.show(io::IO, RS::MatroidRealizationSpace)
    if !RS.representable
        print(io, "The matroid is not representable.")
    else
        println(io, "The representations of the matroid are parametrized by the matrix")
        # println isn't ideal as it prints the matrix as one big line
        display(RS.representation_matrix)
        println(io, "in the ", RS.ambient_ring)
        println(io, "within the vanishing set of the ideal\n", RS.defining_ideal)
        println(io, "avoiding the zero loci of the polynomials\n", RS.inequations)
    end
end

#returns the factors of f, but not the exponents
function poly_2_factors(f::RingElem)
    return collect(keys(Dict(factor(f))))
end

# returns the unique factors of the elements of Sgen, again no exponents. 
function gens_2_factors(Sgens::Vector{<:RingElem})
    return unique!(vcat([poly_2_factors(f) for f in Sgens]...))
end


function stepwise_saturation(I::Ideal, Sgens::Vector{<:RingElem})
    foreach(f -> I = saturation(I,ideal([f])), Sgens)
    return I
end

function realization_bases_coordinates(Bs::Vector{Vector{Int}}, A::Vector{Int})

    d = length(Bs[1])
    B = A[1:d]
    c1 = A[d+1]

    coord_bases = [b for b in Bs if length(symdiff(B,b)) == 2]    
    new_coords = Vector{Vector{Int}}()

    for b in coord_bases
        if is_subset(b, A)
            continue
        end
        
        row_b = setdiff(B,b)[1]
        row_b = count(a->(a<row_b), B) + 1
        
        col_b = setdiff(b,B)[1]
        col_b = col_b - length([a for a in A if a <= col_b])

        push!(new_coords, [row_b, col_b])
    end
    
    return sort!(new_coords, by = x -> (x[2], x[1]))
end


function projective_identity(d::Int)
    if d == 1
        return ones(Int, 1, 1)
    end

    X = zeros(Int, d, d+1)
    for i in 1:d
        X[i, i] = 1
        X[i, d+1] = 1
    end
    return X
end

function interlace_columns(M::MatrixElem{<:RingElem}, N::MatrixElem{<:RingElem}, 
                           A::Vector{Int}, R::AbstractAlgebra.Ring)   
    
    M_nrows, M_ncols = size(M)
    N_nrows, N_ncols = size(N)
    n = M_ncols + N_ncols

    Ac = [i for i in 1:n if !(i in A)]
    
    X = zero_matrix(R, M_nrows, n)
    X[:, A] = M
    X[:, Ac] = N
    
    return X 
end

function partial_matrix_max_rows(Vs::Vector{Vector{Int}})
    nr = maximum([x[1] for x in Vs])
    cols = unique!([x[2] for x in Vs])
    first_nonzero_cols = Dict{Int, Int}(c => maximum(i for i in 1:nr if [i,c] in Vs) for c in cols)
    return first_nonzero_cols 
end

function realization_matrix(Q::Matroid, A::Vector{Int}, F::AbstractAlgebra.Ring)
    d = rank(Q)
    n = length(matroid_groundset(Q))
    Bs = bases(Q)
    RBC = realization_bases_coordinates(Bs, A)
    D = partial_matrix_max_rows(RBC)
    
    numVars = length(RBC) - (n-d-1)
    R, x = polynomial_ring(F, numVars)
    Id = matrix(R, projective_identity(d))
    
    QR = [x for x in RBC if x[1] != D[x[2]]]
    X = zero_matrix(R, d, n-d-1)
    
    k=1
    for j in 1:n-d-1, i in 1:d
        if [i,j] in QR
            X[i,j] = x[k]; k+=1
        elseif(j in keys(D) && i == D[j])
            X[i,j] = R(1)            
        else
            X[i,j] = R(0)
        end
    end
    
    mat = interlace_columns(Id, X, A, R)
    return (R, x, mat)
end

function new_matroid_realization_space(Q::Matroid, A::Vector{Int};
    F::AbstractAlgebra.Ring = ZZ, saturate::Bool=false)::MatroidRealizationSpace

    rk = rank(Q)
    n = length(Q)
    Bs = bases(Q)
    
    polyR, x, mat = realization_matrix(Q, A, F)
    
    eqs = Vector{RingElem}()
    ineqs = Vector{RingElem}()
    
    
    for col in subsets(Vector(1:n),rk)    
        col_det = det(mat[:,col])
        
        if total_degree( col_det ) <= 0 

            if col_det != 0 && col in Bs 
                isunit(col_det) && continue
            elseif col_det != 0 # and col is not a basis
                error("determinant nonzero but set not a basis")
            elseif col in Bs 
                error( "determinant zero but set is a basis" )
            else
                continue
            end
        end
        
        if  col in Bs
            push!(ineqs, col_det)
        else
            push!(eqs, col_det)
        end
    end
    
    def_ideal = ideal(polyR, eqs)
    def_ideal = ideal(groebner_basis(def_ideal))
    ineqs = gens_2_factors(ineqs)
    
    if saturate #|| polyR.nvars < 10
        def_ideal = stepwise_saturation(def_ideal,ineqs)
        def_ideal = ideal(groebner_basis(def_ideal))
    end
    
    representable = !(isone(def_ideal))

    !representable && return MatroidRealizationSpace(def_ideal, nothing, nothing, nothing, false)
    
    return MatroidRealizationSpace(def_ideal, ineqs, polyR, mat, representable)
    
end


new_matroid_realization_space (generic function with 1 method)

In [111]:
R, (x,y,z) = QQ["x", "y", "z"]
Igens = [x+y, x^2]
ideal_vars(Igens) == [x,y]

true

**Function:** `find_solution_v(v::RingElem, Igens::Vector{<:RingElem}, Sgens::Vector{<:RingElem}, R::MPolyRing) `
This function attempts to solve for the variable `v` in the ring $S^{-1}R/I$ where $S$ is the multiplicative semigroup generated by `Sgens`  and $I$ is the ideal generated by `Igens`. 

*Example:* 
```
R, (x,y,z) = QQ["x", "y", "z"]
Igens = [y^3-x^2+1, x^2*y-z^3+6]
Sgens = [x,y,z]
find_solution_v(y, Igens, Sgens, R) == (z^3-6)//x^2
```
meaning that $y \equiv (z^3 - 6 )/ x^2 \mod I$ in  $S^{-1}R$.

---

**Function:** `sub_map(v::RingElem, t::RingElem, R::MPolyRing, xs::Vector{<:RingElem})`

Creates a homomorphism $R \to \mathsf{Frac}(R)$ that sends the variable `v` to `t`.

*Example:* 
```
R, (x,y,z) = QQ["x", "y", "z"]
phi = sub_map(y, (z^3-6)//x^2, R, [x,y,z])
phi.([x,y,z]) == [x,(z^3-6)//x^2,z]
```
---

**Function:** `sub_v(v::RingElem, t::RingElem, f::RingElem, R::AbstractAlgebra.Ring, xs::Vector{<:RingElem})`

In the polynomial `f`, substitutes `v` by `t` and returns the numerator. 

*Example:* 

```
R, (x,y,z) = QQ["x", "y", "z"]
t = (z^3-6)//x^2
f = y^3-x^2+1
g = sub_v(y, t, f, R, [x,y,z])
g == (z^3-6)^3 + x^6*(-x^2 + 1)
```

---

**Function:** `clean(f::RingElem, R::MPolyRing, Sgens::Vector{<:RingElem})`

Removes all factors in `f` that are in `Sgens` (if $S$ is the semigroup generated by `Sgens` then this just removes the factors of `f` that are units in $S^{-1} R$).

*Example:*
```
R, (x,y,z) = QQ["x", "y", "z"]
f = x*y^2*z^3*(x^2+y-x*z^3)
Sgens = [x,y,z]
clean(f, R, Sgens) == x^2+y-x*z^3
```

---

**Function:** `ideal_vars(Igens::Vector{<:RingElem})`

Returns all variables appearing in a poylnomial of `Igens`.

*Example:*
```
R, (x,y,z) = QQ["x", "y", "z"]
Igens = [x+y, x^2]
ideal_vars(Igens) == [x,y]

```

---

**Function:** `n_new_Sgens(x::RingElem, t::RingElem, Sgens::Vector{<:RingElem}, R::AbstractAlgebra.Ring, xs::Vector{<:RingElem})`

Replaces `x` with `t` in the elements of `Sgens`, and takes the unique factors (of the numerators). (The result is supposed to be generators of new semigroup after the substitution `x` to `t`). 

*Example:*


---

**Function:** `n_new_Igens(x::RingElem, t::RingElem, Igens::Vector{<:RingElem}, Sgens::Vector{<:RingElem}, R::AbstractAlgebra.Ring, xs::Vector{<:RingElem}) `

Replaces `x` with `t` in the elements of `Igens`, and removes factors that are in `Sgens`.  (The result is supposed to be generators of new ideal after the substitution `x` to `t`). 


*Example:*

---

**Function:** `matrix_clear_den_in_col(X::Oscar.MatElem, c::Int)`

Given a column index `x` in a matrix `X`, multiplies column `c`  by the lcm of the denominators in that column.  

*Example:*
```
R, (x,y,z) = QQ["x", "y", "z"]
X = matrix(FractionField(R), [x//(z-1) y//(x+1) z; -y+1 x//z 2*x*y ]) 
matrix_clear_den_in_col(X, 2) == matrix(FractionField(R), [x//(z-1) y*z z; -y+1 x^2+x 2*x*y]) 
```

---

**Function:** `matrix_clear_den(X::Oscar.MatElem)`


*Example:*
```
R, (x,y,z) = QQ["x", "y", "z"]
X = matrix(FractionField(R), [x//(z-1) y//(x+1) z; -y+1 x//z 2*x*y ]) 
matrix_clear_den(X) == matrix(FractionField(R), [x y*z z; (-y+1)*(z-1) x^2+x 2*x*y]) 
```
---

**Function:** `reduce_ideal_one_step(MRS::MatroidRealizationSpace, elim::Vector{<:RingElem}, fullyReduced::Bool)`

auxiliary function for `reduce_ideal_full`.  

---

**Function:** `reduce_ideal_full(MRS::MatroidRealizationSpace, elim::Vector{RingElem} = Vector{RingElem}(), fullyReduced::Bool = false) `

Given a `MatroidRealizationSpace`, gives a simplier expression for the ideal by systematically eliminating variables. 

*Example:*
```
MR = new_matroid_realization_space(non_fano_matroid(), [1,2,4,7]; F=QQ)
red_MR = reduce_ideal_full(MR)
R = red_MR.ambient_ring
I = red_MR.defining_ideal
length_S = length(red_MR.inequations)
M = red_MR.representation_matrix
(I,length_S, M) == (ideal(R, [0]), 0, matrix(R, [1 0 1 0 1 0 1; 0 1 1 0 0 1 1; 0 0 0 1 1 1 1]))
```

**Function:** `rank_plus1_circuits(Q::Matroid)`

returns a curcuit of size `rank(Q)` + 1.

In [20]:
function find_solution_v(v::RingElem, Igens::Vector{<:RingElem}, 
                         Sgens::Vector{<:RingElem}, R::MPolyRing) 

    
    with_v_deg_1 = [g for g in Igens if isone(degree(g,v))] 
    length(with_v_deg_1) != 0 || return "can't isolate"

    for f in with_v_deg_1

        den = coefficient_v(v, f)
        fac_den = poly_2_factors(den)
        !issubset(fac_den, Sgens) && continue
        no_v = coeff(f, [v], [0])
        #no_v = [term(f,i) for i in 1:length(f) if !(v in vars(monomial(f,i)))]
        iszero(length(no_v)) && continue
              
        h = R(-1)*sum(no_v)
        return h//den
        
    end
    return "can't solve for v"
end

function sub_map(v::RingElem, t::RingElem, R::MPolyRing, xs::Vector{<:RingElem}) 
    xs_v = map(x -> x==v ? t : x, xs )    
    return hom(R,FractionField(R), a->a, xs_v)
end

# replace v by t in f, only return the numerator.
function sub_v(v::RingElem, t::RingElem, f::RingElem, R::AbstractAlgebra.Ring, xs::Vector{<:RingElem}) 
    m = sub_map(v,t,R,xs) 
    new_f = numerator(m(f))
    return new_f 
end


# removes factors that are in the semigroup generated by Sgens
function clean(f::RingElem, R::MPolyRing, Sgens::Vector{<:RingElem})   
    
    fFactors = factor(f)
    FactorsDict = Dict(fFactors) 
    cleanf_arr = [k^(FactorsDict[k]) for k in keys(FactorsDict) if !(k in Sgens) || is_unit(k)]
    length(cleanf_arr) > 0 ? prod(cleanf_arr) : unit(fFactors)
    
end

#variables in ideal
function ideal_vars(Igens::Vector{<:RingElem}) 
    return unique!(vcat([vars(gen) for gen in Igens]...))
end

function n_new_Sgens(x::RingElem, t::RingElem, Sgens::Vector{<:RingElem}, 
                     R::AbstractAlgebra.Ring, xs::Vector{<:RingElem}) 
    preSgens = unique!([sub_v(x, t, f, R, xs) for f in Sgens])
    return gens_2_factors(preSgens)
end

function n_new_Igens(x::RingElem, t::RingElem, Igens::Vector{<:RingElem}, 
                     Sgens::Vector{<:RingElem}, R::AbstractAlgebra.Ring, xs::Vector{<:RingElem}) 

    preIgens = unique!([clean(sub_v(x, t, f, R, xs), R, Sgens) for f in Igens])
    return filter(x-> x!= R(0), preIgens)
end

function matrix_clear_den_in_col(X::Oscar.MatElem, c::Int)
    Xc = [denominator(f) for f in X[:, c]]
    t = lcm(Xc)
    return multiply_column!(X, t, c)

end

function matrix_clear_den(X::Oscar.MatElem)
    rs, cs = size(X)
    for c in 1:cs
        X = matrix_clear_den_in_col(X, c)
    end
    return X
end


function reduce_ideal_one_step(MRS::MatroidRealizationSpace, 
                               elim::Vector{<:RingElem}, 
                               fullyReduced::Bool)

    Igens = gens(MRS.defining_ideal)
    Sgens = MRS.inequations
    R = MRS.ambient_ring
    FR = FractionField(R)
    xs = gens(R)
    X = MRS.representation_matrix
    nr, nc = size(X)
    
    Ivars = ideal_vars(Igens);

    for x in Ivars 
        t = find_solution_v(x, Igens, Sgens, R)
        t isa String && continue
        
        phi = sub_map(x, t, R, xs)
        
    Sgens_new = n_new_Sgens(x, t, Sgens, R, xs);
        Igens_new = n_new_Igens(x, t, Igens, Sgens_new, R, xs);
        push!(elim, x)
        
        phiX = matrix(FR, [phi(X[i,j]) for i in 1:nr, j in 1:nc  ] )
        nX_FR = matrix_clear_den(phiX)
        nX = matrix(R, [numerator(nX_FR[i,j])  for i in 1:nr, j in 1:nc ])
        
        GBnew = collect(groebner_basis(ideal(R, Igens_new)))         
        MRS_new = MatroidRealizationSpace(ideal(R, GBnew), Sgens_new, R, nX, MRS.representable)
        
        return (MRS_new, elim, fullyReduced)
    end

    return (MRS, elim, true)

end


function reduce_ideal_full(MRS::MatroidRealizationSpace,
                               elim::Vector{RingElem} = Vector{RingElem}(), 
                               fullyReduced::Bool = false) 
    
    output = reduce_ideal_one_step(MRS, elim, fullyReduced)
    output isa String && return "Not Realizable 0 in Semigroup"
    (MRS, elim, fullyReduced) = output    
    
    !fullyReduced && return reduce_ideal_full(MRS, elim, fullyReduced)
    
    R = MRS.ambient_ring
    xs = gens(R)
    cR = coefficient_ring(R)
    X = MRS.representation_matrix
    nr, nc = size(X)
    Igens = gens(MRS.defining_ideal)
    Sgens = MRS.inequations
    
    zero_elim = []        
    for i in 1:length(xs)
        if xs[i] in elim
            push!(zero_elim, 0)
        else
            push!(zero_elim, "x"*string(i) ) 
        end
    end
        
    xnew_str = Vector{String}(filter(x -> x!=0,  zero_elim))    
        
    if length(xnew_str) == 0
        phi = hom(R, cR, a->a, [cR(0) for i in 1:length(xs)])
    else
        Rnew, xnew = polynomial_ring(coefficient_ring(R), length(xnew_str)) 
    
        zero_elim_var = []
        j=1
        for i in 1:length(zero_elim)
            if xs[i] in elim
                push!(zero_elim_var, Rnew(0))
            else
                push!(zero_elim_var, xnew[j] ) 
                j+=1
            end
        end
        
        phi = hom(R, Rnew, a->a, zero_elim_var)
    
    end
    
    ambR = codomain(phi)
    Inew = ideal(ambR, phi.(Igens))
    Sgens_new = phi.(Sgens)
    
    normal_Sgens = gens_2_factors([normal_form(g, Inew) for g in Sgens_new])
    unique!(normal_Sgens)


    Xnew = matrix(ambR, [phi(X[i,j]) for i in 1:nr, j in 1:nc])

    MRS_new = MatroidRealizationSpace(Inew, normal_Sgens, ambR, Xnew, MRS.representable)

    return MRS_new
end

function rank_plus1_circuits(Q::Matroid)
    rk = rank(Q) 
    mc = [c for c in circuits(Q) if length(c) == r+1]
    return mc[1]
end

reduce_ideal_full (generic function with 3 methods)

### File: jacobianCriterion.final.jl

---

**Function:** `jacobian_matrix(R::MPolyRing, x::Vector{<: MPolyElem}, Igens::Vector{MPolyElem})`

Computes the Jacobian matrix of the polynomials in `Igens` with respect to the variables in `x`.

*Example:*
```
R, (x,y,z) = QQ["x", "y", "z"]
jacobian_matrix(R, [x,y,z], gens(I))
```
---

**Function:** `singular_locus(R::MPolyRing, x::Vector{<:MPolyElem}, I, Sgens, do_saturation = false)`

Computes the singular locus of $X = \mathsf{Spec}(S^{-1}R/I)$ where `R` is a polynomial ring. Note: this does not work if $X$ is not of pure codimension. 

*Example:* 
```
R, (x,y,z) = QQ["x", "y", "z"]

I = ideal(R, [y^2-x^3-x^2, x+y-z])
Sing = new_singular_locus(R, [x,y,z], I, Vector{RingElem}())
Sing == ideal(R, [x,y,z])

I = ideal(R, [y^2-x^3-x^2])
Sing = new_singular_locus(R, [x,y,z], I, Vector{RingElem}())
Sing == ideal(R, [x,y])

I1 = ideal(R, [x+y, y+z]); I2 = ideal(R, [z^2-x*y])
I = intersect(I1,I2) # a line union a quadric surface. 
new_singular_locus(R, [x,y,z], I, [x,y,z]) == "Not pure codimension"
```

In [141]:
function jacobian_matrix(R::MPolyRing, x::Vector{<: MPolyElem}, Igens::Vector{<:MPolyElem})
    return matrix(R, [derivative(f,j) for f in Igens, j in 1:length(x) ])
end

function new_singular_locus(R::MPolyRing, x::Vector{<:MPolyElem}, I, Sgens, do_saturation = false) 
    
    if do_saturation
        I = stepwise_saturation(I, Sgens)
    end
        
    m_primes = minimal_primes(I); 
    codim_m_primes = unique!(codim.(m_primes)); 
    
    if length(codim_m_primes) > 1
        return "Not pure codimension"
    end
    c=codim_m_primes[1]
    if c > length(x)
        return ideal([R(1)])
    end
    Igens = gens(I); 
    Jmat = jacobian_matrix(R, x, Igens) ; 
    
    Sing = I + ideal(minors(Jmat, c))
    
    return stepwise_saturation(Sing, Sgens)
end

new_singular_locus (generic function with 2 methods)